In [1]:
import os
import sys
import json
import ndex2
from ndex2.nice_cx_network import NiceCXNetwork
from ndex2.nice_cx_network import DefaultNetworkXFactory
import networkx as nx
import requests

In [2]:
network =  ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='38a9cebf-23dc-11e8-b939-0ac135e8bacf') # (Nodes1127)

In [3]:
tmp=set()
for e in network.get_edges():
    tmp.add(e[1]['s'])
    tmp.add(e[1]['t'])
tmp_dict={}
for i,j in enumerate(tmp):
    tmp_dict.update({j:i})

In [4]:
edge_data=""
for e in network.get_edges():
    edge_data+=str(tmp_dict[e[1]['s']])+"\t"+str(tmp_dict[e[1]['t']])+"\n"

In [5]:
template={
          "algorithm": "louvain",
          "graphdirected": "false",
          "data": "",
          "ipAddress": "string",
          "customParameters": {
            "additionalProp1": "string",
            "additionalProp2": "string",
            "additionalProp3": "string"
          }
        }

In [6]:
template['data']=edge_data

In [7]:
res = requests.post('http://ddot.ucsd.edu/cd/communitydetection/v1',json=template)
print('Created task. ID: {}'.format(res.json()["id"]))

Created task. ID: 7fc0fc13-5c7a-40f3-9ce9-2c5c86ef79a4


In [8]:
res = requests.get("http://ddot.ucsd.edu/cd/communitydetection/v1/"+res.json()['id'])
res.json()['status']

'complete'

In [9]:
if(res.json()['status']!='complete'):
    print("*********    Calculating!!!!    Wait!!!!!     *********\n\n\n")

In [10]:
import sys
import json
import ndex2
from auto_graph_visualizer.utils import *
from auto_graph_visualizer.visualizer import AutoGraphVisualizer

In [11]:
options = {
        "graph_name": "test_network",
        "output_file_name": "test_out",
        "algorithm": "rest",
        "color_palette": "hls",
        "nodesize": "betweenness",
        "maxnodesize": 100,
        "density": "normal",
        "positions": "fa",
        "displaylabels":20
    }



In [12]:
viz = AutoGraphVisualizer(options,res.json()['result'])

In [13]:
cx_network = network.to_cx() 
    
cxobj = viz.generate_viz(cx_network)

Generating CX
Name: BioGRID: Protein-Protein Interactions (HIV-1)
Nodes: 1136
Edges: 1324
Node Attributes: 3323
Edge Attributes: 9268



100%|██████████| 2000/2000 [00:32<00:00, 62.39it/s]


BarnesHut Approximation  took  8.58  seconds
Repulsion forces  took  20.51  seconds
Gravitational forces  took  0.24  seconds
Attraction forces  took  0.03  seconds
AdjustSpeedAndApplyForces step  took  1.32  seconds
[7262.468978689562, 8295.732502741093, 8404.682072407177, 8404.682072407177, 8404.682072407177, 9781.867007308598, 9781.867007308598, 9781.867007308598, 12871.4814947476, 16870.798610889502, 24617.325025877468, 48738.173828199164, 59507.38703944492, 71122.62603284113, 91086.01650659232, 116561.41613606711, 144358.90191637765, 186562.2106962277, 192783.91515256002, 276280.57042694866]
Generating CX


In [14]:
from py2cytoscape import cyrest

# This should be changed to your host IP
host_ip = '192.168.254.19'
cytoscape=cyrest.cyclient(host=host_ip)
cytoscape.version()
cytoscape.status()

apiVersion v1
cytoscapeVersion 3.7.1
CyREST online!


In [15]:
cytoscape.session.new()

In [16]:
import requests

cyrest_url = 'http://' + host_ip + ':1234/cyndex2/v1/networks/cx'
print(cyrest_url)
response = requests.post(cyrest_url, data=json.dumps(cxobj))
print(response.json())

http://192.168.254.19:1234/cyndex2/v1/networks/cx
{'data': {'suid': 4986, 'uuid': ''}, 'errors': []}
